In [1]:
!pip3 -q install python-dotenv langchain

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo",streaming=True)


In [4]:
# llm.invoke("What will be the weather in munich today?")

AIMessage(content="I'm sorry, I cannot provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Munich today.", response_metadata={'finish_reason': 'stop'}, id='run-06bd8257-3069-479f-94b4-7cb67f64336a-0')

In [5]:
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.pydantic_v1 import BaseModel, Field

class WInput(BaseModel):
    Name: str = Field(description="Name of the city")
    
class OSAInput(BaseModel):
    Name: str = Field(description="Name of the city")
        
@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city."""
    return "Sunny, 22°C"

@tool
def outdoor_seating_availability(city: str) -> str:
    """Check if outdoor seating is available at a specified restaurant in a given city."""
    return "Outdoor seating is available."


In [6]:
tools = [fake_weather_api,outdoor_seating_availability]

In [7]:
llm_with_tools = llm.bind_tools(tools)

In [8]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result


AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_M31MNzdRBbPwEquyR5PxdmN8', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-d3dbf6e2-56aa-459d-a6d9-15264b25f7ac-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_M31MNzdRBbPwEquyR5PxdmN8'}])

In [9]:
result = llm_with_tools.invoke("How will the weather be in munich today? I would like to eat outside if possible")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_QwTWTs08OlIngkTA8gyYeWJ9', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'index': 1, 'id': 'call_PDQSLz4Orf1rScXhGyKe5bvO', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-642cfb0a-5389-4021-9ea4-13790c6bc9dc-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_QwTWTs08OlIngkTA8gyYeWJ9'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'Munich'}, 'id': 'call_PDQSLz4Orf1rScXhGyKe5bvO'}])

In [10]:
result.tool_calls


[{'name': 'fake_weather_api',
  'args': {'city': 'Munich'},
  'id': 'call_QwTWTs08OlIngkTA8gyYeWJ9'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'Munich'},
  'id': 'call_PDQSLz4Orf1rScXhGyKe5bvO'}]

In [30]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage("How will the weather be in munich today? I would like to eat outside if possible.")]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)


In [31]:
tool_mapping = {"fake_weather_api": fake_weather_api, "outdoor_seating_availability": outdoor_seating_availability}

In [32]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))


In [33]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible.'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_HH9eR3ZL5REJ7rqAAhLkbZ98', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'index': 1, 'id': 'call_eyGii4nMZ3DDEZf3xL6fc9Io', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-9ae62960-50c9-4097-ad67-c78b27b89d5e-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_HH9eR3ZL5REJ7rqAAhLkbZ98'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'Munich'}, 'id': 'call_eyGii4nMZ3DDEZf3xL6fc9Io'}]),
 ToolMessage(content='Sunny, 22°C', tool_call_id='call_HH9eR3ZL5REJ7rqAAhLkbZ98'),
 ToolMessage(content='Outdoor seating is available.', tool_call_id='call_eyGii4nMZ3DDEZf3xL6fc9Io')]

In [34]:
for ch in llm_with_tools.stream(messages):
    print(ch.content,end="",flush=True)

The weather in Munich today is sunny with a temperature of 22°C. Outdoor seating is available, so you can enjoy your meal outside.